In [86]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score

### 0. 필요한 데이터 추출

In [74]:
D_ITEMS = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ITEMS.csv")
df = pd.read_csv("~/project/MIMIC-III/Data/SAPS-II/SAPS_Feature.csv")

/tmp/ipykernel_3619427/2086009227.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("~/project/MIMIC-III/Data/SAPS-II/SAPS_Feature.csv")


In [75]:
D_ITEMS[D_ITEMS['LABEL'].str.contains('Mean Airway',na=False)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
673,410,444,Mean Airway Pressure,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10059,13251,224697,Mean Airway Pressure,Mean Airway Pressure,metavision,chartevents,Respiratory,cmH2O,Numeric,NaN


In [76]:
# item_list = [444]
# item_list = [224697]
item_list = [444, 224697]
df1 = df[df['ITEMID'].isin(item_list)]

df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUE': lambda x: np.min(x.astype(float))})
df2

# df2.to_csv("~/project/MIMIC-III/Data/airway_pressure.csv", index=False)

/tmp/ipykernel_3619427/1747327846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0, inplace=True)  # 결측치를 0으로 변경


,SUBJECT_ID,HADM_ID,ITEMID,VALUE
0,3,145834,444,7.2
1,9,150750,444,7.0
2,12,112213,444,7.0
3,13,143045,444,0.0
4,17,161087,444,8.0
...,...,...,...,...
24800,99939,159023,224697,8.0
24801,99955,108494,224697,5.0
24802,99982,151454,224697,7.0
24803,99985,176670,224697,7.0


In [77]:
D_ITEMS[D_ITEMS['LABEL'].str.contains('SpO2',na=False)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
1418,599,646,SpO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2469,1923,5820,SpO2 Alarm [Low],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4804,4998,8554,SpO2 Alarm [High],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5080,4343,6719,SpO2-L,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
11926,14442,226253,SpO2 Desat Limit,SpO2 Desat Limit,metavision,chartevents,Alarms,%,Numeric,NaN


In [78]:
item_list = [646]
df1 = df[df['ITEMID'].isin(item_list)]

df1.fillna(0, inplace=True)  # 결측치를 0으로 변경
df2 = df1.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUE': lambda x: np.min(x.astype(float))})
df2

# df2.to_csv("~/project/MIMIC-III/Data/SpO2.csv", index=False)

/tmp/ipykernel_3619427/2781079197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.fillna(0, inplace=True)  # 결측치를 0으로 변경


,SUBJECT_ID,HADM_ID,ITEMID,VALUE
0,3,145834,646,74.0
1,4,185777,646,91.0
2,6,107064,646,84.0
3,9,150750,646,90.0
4,11,194540,646,93.0
...,...,...,...,...
27211,32805,168079,646,69.0
27212,32805,169344,646,90.0
27213,32807,140728,646,92.0
27214,32809,101254,646,91.0


### 1. Respiratory 겹치는 환자

In [79]:
df1 = pd.read_csv("~/project/MIMIC-III/Data/SAPS-II/6-3. saps2_PaO2.csv")
df2 = pd.read_csv("~/project/MIMIC-III/Data/SAPS-II/6-4. saps2_FiO2.csv")
df3 = pd.read_csv("~/project/MIMIC-III/Data/SpO2.csv")
df4 = pd.read_csv("~/project/MIMIC-III/Data/airway_pressure.csv")
s_id = pd.read_csv("~/project/MIMIC-III/Data/SUBJECTID.csv")


df5 = pd.merge(df1, df2, on=['SUBJECT_ID', 'HADM_ID'])
df5.drop('ITEMID_x', axis=1, inplace=True)
df5.drop('ITEMID_y', axis=1, inplace=True)
df5.rename(columns={'VALUE_x':'PaO2', 'VALUE_y':'FiO2', 'VALUE':'SpO2'}, inplace=True)

df5 = pd.merge(df5, df3, on=['SUBJECT_ID', 'HADM_ID'])
df5.drop('ITEMID', axis=1, inplace=True)
df5.rename(columns={'VALUE':'SpO2'}, inplace=True)

df5 = pd.merge(df5, df4, on=['SUBJECT_ID', 'HADM_ID'])
df5.drop('ITEMID', axis=1, inplace=True)
df5.rename(columns={'VALUE':'AIRWAY'}, inplace=True)
df5 = df5.fillna(0)
df5

,SUBJECT_ID,HADM_ID,PaO2,FiO2,SpO2,AIRWAY
0,3,145834,176.000000,0.588889,74.0,7.2
1,9,150750,90.333333,0.657143,90.0,7.0
2,12,112213,206.857143,0.477778,0.0,7.0
3,13,143045,370.400000,0.750000,94.0,0.0
4,17,161087,171.500000,0.625000,96.0,8.0
...,...,...,...,...,...,...
11652,32800,179038,193.400000,0.600000,97.0,8.0
11653,32805,124834,86.600000,0.700000,0.0,0.0
11654,32807,140728,319.000000,0.750000,92.0,7.0
11655,32810,189751,154.400000,0.655556,74.0,6.0


In [80]:
# 겹치는 환자
df6 = pd.merge(df5, s_id[['SUBJECT_ID', 'HADM_ID']], on=['SUBJECT_ID', 'HADM_ID'], how='inner')
df6

,SUBJECT_ID,HADM_ID,PaO2,FiO2,SpO2,AIRWAY
0,94,140037,153.125000,0.666667,79.0,2.0
1,101,175533,115.428571,0.566667,67.0,2.0
2,111,192123,153.800000,0.518182,89.0,0.0
3,141,168006,129.857143,0.540000,0.0,10.0
4,148,199488,96.000000,1.000000,0.0,4.0
...,...,...,...,...,...,...
2680,32781,142684,245.800000,0.590000,88.0,0.0
2681,32786,165351,143.500000,0.785714,86.0,0.0
2682,32790,167494,324.000000,0.528571,84.0,5.0
2683,32800,179038,193.400000,0.600000,97.0,8.0


### 2. OSI&OI score

In [81]:
def calculate_OI(df):
    df['OI'] = df['PaO2'] * df['AIRWAY'] * 100 / df['FiO2']
    return df

def calculate_OSI(df):
    df['OSI'] = df['SpO2'] * df['AIRWAY'] * 100 / df['FiO2']
    return df

# OI와 OSI 계산을 위한 함수 호출
df6 = calculate_OI(df6)
df6 = calculate_OSI(df6)
df6

,SUBJECT_ID,HADM_ID,PaO2,FiO2,SpO2,AIRWAY,OI,OSI
0,94,140037,153.125000,0.666667,79.0,2.0,45937.500000,23700.000000
1,101,175533,115.428571,0.566667,67.0,2.0,40739.495370,23647.058575
2,111,192123,153.800000,0.518182,89.0,0.0,0.000000,0.000000
3,141,168006,129.857143,0.540000,0.0,10.0,240476.191538,0.000000
4,148,199488,96.000000,1.000000,0.0,4.0,38400.000000,0.000000
...,...,...,...,...,...,...,...,...
2680,32781,142684,245.800000,0.590000,88.0,0.0,0.000000,0.000000
2681,32786,165351,143.500000,0.785714,86.0,0.0,0.000000,0.000000
2682,32790,167494,324.000000,0.528571,84.0,5.0,306486.485005,79459.459075
2683,32800,179038,193.400000,0.600000,97.0,8.0,257866.666026,129333.333012


In [82]:
# OI score 0 아닌 경우
OI = df6[df6["OI"] != 0]
OI

,SUBJECT_ID,HADM_ID,PaO2,FiO2,SpO2,AIRWAY,OI,OSI
0,94,140037,153.125000,0.666667,79.0,2.0,45937.500000,23700.000000
1,101,175533,115.428571,0.566667,67.0,2.0,40739.495370,23647.058575
3,141,168006,129.857143,0.540000,0.0,10.0,240476.191538,0.000000
4,148,199488,96.000000,1.000000,0.0,4.0,38400.000000,0.000000
5,164,182743,93.000000,0.800000,26.0,6.0,69750.000693,19500.000194
...,...,...,...,...,...,...,...,...
2674,32737,151135,165.000000,0.610000,61.0,7.0,189344.261555,69999.999726
2677,32746,124485,119.000000,0.561538,89.0,7.0,148342.464300,110945.204392
2679,32779,133757,203.000000,0.412500,0.0,11.0,541333.326489,0.000000
2682,32790,167494,324.000000,0.528571,84.0,5.0,306486.485005,79459.459075


In [83]:
OSI = df6[df6["OSI"] != 0]
OSI

,SUBJECT_ID,HADM_ID,PaO2,FiO2,SpO2,AIRWAY,OI,OSI
0,94,140037,153.125000,0.666667,79.0,2.0,45937.500000,23700.000000
1,101,175533,115.428571,0.566667,67.0,2.0,40739.495370,23647.058575
5,164,182743,93.000000,0.800000,26.0,6.0,69750.000693,19500.000194
6,170,175350,183.000000,0.411111,96.0,6.0,267081.077639,140108.106302
7,179,161310,97.666667,0.845833,85.0,2.0,23093.595978,20098.522097
...,...,...,...,...,...,...,...,...
2673,32697,100941,55.500000,0.500000,83.0,7.0,77700.000000,116200.000000
2674,32737,151135,165.000000,0.610000,61.0,7.0,189344.261555,69999.999726
2677,32746,124485,119.000000,0.561538,89.0,7.0,148342.464300,110945.204392
2682,32790,167494,324.000000,0.528571,84.0,5.0,306486.485005,79459.459075


In [84]:
OI_OSI = df6[(df6['OI'] != 0) & (df6['OSI'] != 0)]
OI_OSI

,SUBJECT_ID,HADM_ID,PaO2,FiO2,SpO2,AIRWAY,OI,OSI
0,94,140037,153.125000,0.666667,79.0,2.0,45937.500000,23700.000000
1,101,175533,115.428571,0.566667,67.0,2.0,40739.495370,23647.058575
5,164,182743,93.000000,0.800000,26.0,6.0,69750.000693,19500.000194
6,170,175350,183.000000,0.411111,96.0,6.0,267081.077639,140108.106302
7,179,161310,97.666667,0.845833,85.0,2.0,23093.595978,20098.522097
...,...,...,...,...,...,...,...,...
2673,32697,100941,55.500000,0.500000,83.0,7.0,77700.000000,116200.000000
2674,32737,151135,165.000000,0.610000,61.0,7.0,189344.261555,69999.999726
2677,32746,124485,119.000000,0.561538,89.0,7.0,148342.464300,110945.204392
2682,32790,167494,324.000000,0.528571,84.0,5.0,306486.485005,79459.459075


### 3. LR

In [85]:
data = pd.read_csv("~/project/MIMIC-III/Data/SUBJECTID_y.csv")
OI_OSI = pd.merge(OI_OSI, data, on=['SUBJECT_ID', 'HADM_ID'], how='left')
result = OI_OSI[['SUBJECT_ID', 'HADM_ID', 'OI', 'OSI', 'y']]
result

,SUBJECT_ID,HADM_ID,OI,OSI,y
0,94,140037,45937.500000,23700.000000,0
1,101,175533,40739.495370,23647.058575,1
2,164,182743,69750.000693,19500.000194,1
3,170,175350,267081.077639,140108.106302,0
4,179,161310,23093.595978,20098.522097,0
...,...,...,...,...,...
1648,32697,100941,77700.000000,116200.000000,0
1649,32737,151135,189344.261555,69999.999726,1
1650,32746,124485,148342.464300,110945.204392,0
1651,32790,167494,306486.485005,79459.459075,0


In [91]:
# OI LR
X = result[['OI']]
y = result['y']

# Train, Test, Validation dataset split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print("OI score with Logistic Regression")
print("Train: ", len(X_train), "Validation: ", len(X_val), "Test: ", len(X_test), "\n ")

# Logistic Regression model creation and hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100]}  

logreg = LogisticRegression(random_state=42)
grid_search = GridSearchCV(logreg, param_grid, cv=3)
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_, "\n")

# Validation Set - Check performance of the best model
best_logreg = grid_search.best_estimator_
y_val_pred = best_logreg.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy, "\n")

# Test dataset - Check performance of the best model
y_pred = best_logreg.predict(X_test)
y_prob = best_logreg.predict_proba(X_test)[:, 1]  # Probability of class 1
test_accuracy = accuracy_score(y_test, y_pred)
test_conf_matrix = confusion_matrix(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_prob)  # AUROC calculation

print("Test Accuracy:", test_accuracy)
print("Test AUROC:", test_roc_auc)
print("Test Confusion Matrix:")
print(test_conf_matrix)

OI score with Logistic Regression
Train:  1057 Validation:  265 Test:  331 
 
Best Hyperparameters: {'C': 0.1} 

Validation Accuracy: 0.7320754716981132 

Test Accuracy: 0.7129909365558912
Test AUROC: 0.4739741302408564
Test Confusion Matrix:
[[236   0]
 [ 95   0]]


In [92]:
# OSI RF
X = result[['OSI']]
y = result['y']

# Train, Test, Validation dataset split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print("OSI score with Logistic Regression")
print("Train: ", len(X_train), "Validation: ", len(X_val), "Test: ", len(X_test), "\n ")

# Logistic Regression model creation and hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100]}  

logreg = LogisticRegression(random_state=42)
grid_search = GridSearchCV(logreg, param_grid, cv=3)
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_, "\n")

# Validation Set - Check performance of the best model
best_logreg = grid_search.best_estimator_
y_val_pred = best_logreg.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy, "\n")

# Test dataset - Check performance of the best model
y_pred = best_logreg.predict(X_test)
y_prob = best_logreg.predict_proba(X_test)[:, 1]  # Probability of class 1
test_accuracy = accuracy_score(y_test, y_pred)
test_conf_matrix = confusion_matrix(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_prob)  # AUROC calculation

print("Test Accuracy:", test_accuracy)
print("Test AUROC:", test_roc_auc)
print("Test Confusion Matrix:")
print(test_conf_matrix)

OSI score with Logistic Regression
Train:  1057 Validation:  265 Test:  331 
 
Best Hyperparameters: {'C': 0.1} 

Validation Accuracy: 0.7320754716981132 

Test Accuracy: 0.7129909365558912
Test AUROC: 0.5836083853702052
Test Confusion Matrix:
[[236   0]
 [ 95   0]]
